In [1]:
get_ipython().run_line_magic('pylab', 'inline')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#from tinc import *
import tinc
import os
import numpy as np
import djlib.mc as mc


In [3]:
#Enlarge the jupyter notebook so that plots are easier to see 
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:


mu = tinc.Parameter("mu", "chemical_potential")
mu.values = np.linspace(-1,1,81)

graph_buffer = tinc.DiskBufferImage("graph", "out.png", "graph_output")

In [5]:
ps = tinc.ParameterSpace("ps")
ps.register_parameters([mu])
ps.enable_cache()

Validating json with /home/derick/anaconda3/envs/py37/lib/python3.7/site-packages/tinc/tinc_cache_schema.json


In [6]:
def make_graph(mu):
    
    t_high = 2000
    t_low = 20
    user="jonny"
    #hard-coding this now, change later
    if(user=="jonny"):
        print("using Jonny's paths")
        plots_dir = '/home/jonnyli/Desktop/CASM/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/plots'
        t_const_right_dir = "/home/jonnyli/Desktop/CASM/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/t_const/mu_3.0_0_0.005_T_2000"
        t_const_left_dir = "/home/jonnyli/Desktop/CASM/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/t_const/mu_-3.0_0_0.005_T_2000"
        heating_dir = "/home/jonnyli/Desktop/CASM/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/heating"
        cooling_dir = "/home/jonnyli/Desktop/CASM/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/cooling"
        #lte_dir = ("/home/jonnyli/Desktop/CASM/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/lte")    
    else:
        print("using Derick's paths")
        plots_dir = '/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/plots't_const_right_dir = "/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/t_const/mu_3.0_0_0.005_T_2000"
        t_const_left_dir = "/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/t_const/mu_-3.0_0_0.005_T_2000"
        heating_dir = "/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/heating"
        cooling_dir = "/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/cooling"
        #lte_dir = ("/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/lte")

    heating_run_name = os.path.join(heating_dir, 'mu_%.4f_%.4f_T_%d_%d' % ((mu), (mu), t_low, t_high))
    cooling_run_name = os.path.join(cooling_dir, 'mu_%.4f_%.4f_T_%d_%d' % ((mu), (mu), t_high, t_low) )
    fig = False

    if os.path.isfile(os.path.join(heating_run_name, 'results.json')) and os.path.isfile(os.path.join(cooling_run_name, 'results.json')):    
        t_const_left = mc.constant_t_run(t_const_left_dir)
        t_const_right = mc.constant_t_run(t_const_right_dir)
        #lte_run = mc.lte_run(lte_dir)
        heating_run = mc.heating_run_no_lte(heating_run_name)
        if mu < 0:
            cooling_run = mc.cooling_run(cooling_run_name, t_const_left)
        else:
            cooling_run = mc.cooling_run(cooling_run_name, t_const_right)
        
        fig = mc.plot_heating_and_cooling(heating_run, cooling_run)
        fig.set_size_inches(20, 10)

    if fig:
        image_path = os.path.join(plots_dir, 'heating_vs_cooling_mu%.4f.png' % (mu) )
        fig.savefig(image_path)
        fig.show()
    
    
    #Path to pre-made images, using scripts from vasp_scripts
    file = open(image_path, 'rb')
    return file.read()


def value_changed(value):
    imagedata = ps.run_process(make_graph)
    graph_buffer.data = imagedata

mu.register_callback(value_changed)

In [7]:
#print(A.values,B.values,kT.values)

#currently not using this display approach
from ipywidgets import GridspecLayout
grid = GridspecLayout(3, 2, height='300px')
grid[:, 1] = graph_buffer.interactive_widget()
grid[0, 0] = A.interactive_widget()
grid[1, 0] = B.interactive_widget()
grid[2, 0] = kT.interactive_widget()
grid


In [8]:
from ipywidgets import Button, HBox, VBox

box = VBox([mu.interactive_control(), graph_buffer.interactive_widget()])
graph_buffer._interactive_widget.width = "100%"
box